# Default strategies insertion into MongoDB

In [1]:
## Libraries
import os
import pkgutil
from importlib import import_module
import inspect
from backtesting import Strategy

from pymongo import MongoClient

### Database connection

In [2]:
## DATABASE
# Establish a connection to MongoDB
client = MongoClient('mongodb://localhost:27017/')

# Access the collection for the strategies
db = client['strategies']

In [3]:
# Delete the existing items in the collection to start fresh
db['default_strategies'].delete_many({})

### Strategy classes definition

In [19]:
# Get the directory where the strategy modules are located
modules_dir = 'modules'

# Create an empty dictionary to store the strategy classes
strategy_classes = {}

# Iterate over the strategy modules in the directory
for _, module_name, _ in pkgutil.iter_modules([modules_dir]):
    # Import the strategy module dynamically
    module = import_module(f'{modules_dir}.{module_name}')

    # Get the strategy class from the module
    strategy_class = next((cls for _, cls in inspect.getmembers(module, inspect.isclass) if issubclass(cls, Strategy)), None)

    # Add the strategy class to the dictionary with the module name as key
    if strategy_class is not None:
        strategy_classes[module_name] = strategy_class

In [20]:
strategy_classes

{'AverageDirectionalMovement': modules.AverageDirectionalMovement.AverageDirectionalMovement,
 'BollingerBands': modules.BollingerBands.BollingerBands,
 'MeanReversion': modules.MeanReversion.MeanReversion,
 'MeanReversionBollinger': modules.MeanReversionBollinger.MeanReversionBollinger,
 'MomentumVolatility': modules.MomentumVolatility.MomentumVolatility,
 'MovingAverageConvergenceDivergence': modules.MovingAverageConvergenceDivergence.MovingAverageConvergenceDivergence,
 'MovingAverageCrossover': modules.MovingAverageCrossover.MovingAverageCrossover,
 'RelativeStrengthIndex': modules.RelativeStrengthIndex.RelativeStrengthIndex,
 'StochasticOverboughtOversold': modules.StochasticOverboughtOversold.StochasticOverboughtOversold,
 'Turtle': backtesting.backtesting.Strategy,
 'VolatilityBreakout': backtesting.backtesting.Strategy}

### Description of the strategies

In [21]:
# Create a dictionary with the descriptions of each strategy
strategy_descriptions = {
    'AverageDirectionalMovement': '''
        The Average Directional Movement (ADX) strategy is a trend-following strategy that uses the ADX indicator
        to identify strong trending markets. The strategy generates a buy signal when the ADX crosses above a threshold
        value, indicating the start of a new trend. It generates a sell signal when the ADX falls below the threshold,
        suggesting a weakening trend or a range-bound market.
    ''',
    'BollingerBands': '''
        The Bollinger Bands strategy is a mean-reversion strategy that uses the Bollinger Bands indicator to identify
        overbought and oversold conditions. The strategy generates a buy signal when the price crosses below the
        lower Bollinger Band and a sell signal when the price crosses above the upper Bollinger Band.
    ''',
    'MeanReversion': '''
        The Mean Reversion strategy is a mean-reversion trading strategy that identifies potential price reversals.
        It calculates the z-score of the current closing price based on the mean and standard deviation over a
        specified lookback period. When the z-score exceeds a certain threshold (positive or negative), indicating
        an extreme deviation from the mean, it generates a sell or buy signal, respectively.
    ''',
    'MeanReversionBollinger': '''
        The Mean Reversion Bollinger strategy is a mean-reversion strategy that combines the concepts of mean reversion
        and Bollinger Bands. It identifies potential price reversals by calculating the z-score of the current closing
        price based on the mean and standard deviation over a specified lookback period. When the z-score exceeds a
        certain threshold (positive or negative), indicating an extreme deviation from the mean, it generates a sell
        or buy signal, respectively. The threshold is determined by the number of standard deviations from the mean.
    ''',
    'MomentumVolatility': '''
        The Momentum Volatility strategy is a momentum-based strategy that combines momentum and volatility indicators.
        It generates a sell signal when the momentum indicator is positive and the average true range (ATR) exceeds
        a specified threshold. It generates a buy signal when the momentum indicator is negative and the ATR exceeds
        the threshold.
    ''',
    'MovingAverageConvergenceDivergence': '''
        The Moving Average Convergence Divergence (MACD) strategy is a trend-following strategy that uses the MACD
        indicator to identify potential buy and sell signals. It generates a buy signal when the MACD histogram
        crosses above zero and a sell signal when the MACD histogram crosses below zero.
    ''',
    'MovingAverageCrossover': '''
        The Moving Average Crossover strategy is a trend-following strategy that uses two moving averages of different
        periods to generate buy and sell signals. It generates a buy signal when the shorter-term moving average
        crosses above the longer-term moving average and a sell signal when the shorter-term moving average crosses
        below the longer-term moving average.
    ''',
    'RelativeStrengthIndex': '''
        The Relative Strength Index (RSI) strategy is a momentum-based strategy that uses the RSI indicator to identify
        overbought and oversold conditions. It generates a sell signal when the RSI crosses above a specified sell
        threshold and a buy signal when the RSI crosses below a specified buy threshold.
    ''',
    'StochasticOverboughtOversold': '''
        The Stochastic Overbought/Oversold strategy is a momentum-based strategy that uses the Stochastic Oscillator
        to identify overbought and oversold conditions. It generates a buy signal when the Stochastic Oscillator (%K)
        falls below the oversold threshold and then crosses above it, and a sell signal when it rises above the
        overbought threshold and then crosses below it.
    ''',
    'Turtle': '''
        The Turtle strategy is a trend-following strategy based on breakout levels. It identifies potential entry points
        by detecting breakouts above the highest high or below the lowest low over a specified lookback period. Once a
        breakout occurs, it enters a long or short position accordingly. The strategy then waits for a specified number
        of bars to elapse before checking for exit conditions, which include prices crossing back below the lowest low
        (for long positions) or above the highest high (for short positions).
    ''',
    'VolatilityBreakout': '''
        The Volatility Breakout strategy is a trend-following strategy that aims to capture price breakouts based on
        recent volatility. It calculates the highest high and lowest low over a specified lookback period and adjusts
        them by a volatility factor. When the current closing price breaks above the adjusted highest high, it generates
        a buy signal. Conversely, when the current closing price breaks below the adjusted lowest low, it generates
        a sell signal.
    '''
}

### Description of the tunable parameters

In [22]:
# Create a dictionary with the parameters and descriptions of each strategy
parameter_descriptions = {
    "AverageDirectionalMovement": {
        'adx_period': {
            'value': 7,
            'description': 'The number of periods for calculating the Average Directional Movement Index (ADX). It measures the strength of a trend. If the ADX exceeds the threshold, it indicates a strong trend and triggers buying.'
        },
        'adx_threshold': {
            'value': 15,
            'description': 'The threshold value for the ADX to trigger buying. If the ADX exceeds this value, it indicates a strong trend and triggers buying.'
        }
    },
    "BollingerBands": {
        'period': {
            'value': 30,
            'description': 'The number of periods for calculating the Bollinger Bands. It determines the lookback period for calculating the moving average and standard deviation.'
        },
        'std_devs': {
            'value': 3,
            'description': 'The number of standard deviations for calculating the Bollinger Bands. It determines the width of the bands and the threshold for overbought and oversold conditions.'
        }
    },
    "MeanReversion": {
        'lookback_period': {
            'value': 21,
            'description': 'The number of periods used to calculate the mean and standard deviation for the mean reversion strategy. It determines the lookback period for calculating the z-score.'
        },
        'z_score_threshold': {
            'value': 3.0,
            'description': 'The threshold value for the z-score to trigger buying or selling. If the z-score exceeds this value (positive or negative), it indicates an extreme deviation from the mean and triggers a buy or sell signal.'
        }
    },
    "MeanReversionBollinger": {
        'lookback_period': {
            'value': 40,
            'description': 'The number of periods used to calculate the mean and standard deviation for the Mean Reversion Bollinger strategy. It determines the lookback period for calculating the z-score.'
        },
        'z_score_threshold': {
            'value': 3,
            'description': 'The threshold value for the z-score to trigger buying or selling in the Mean Reversion Bollinger strategy. If the z-score exceeds this value (positive or negative), it indicates an extreme deviation from the mean and triggers a buy or sell signal.'
        }
    },
    "MomentumVolatility": {
        'lookback_period': {
            'value': 10,
            'description': 'The number of periods for calculating the momentum indicator in the Momentum Volatility strategy. It determines the lookback period for calculating momentum.'
        },
        'atr_period': {
            'value': 15,
            'description': 'The number of periods for calculating the Average True Range (ATR) in the Momentum Volatility strategy. It determines the lookback period for measuring volatility.'
        },
        'atr_threshold': {
            'value': 2.5,
            'description': 'The threshold value for the ATR to trigger buying or selling in the Momentum Volatility strategy. If the ATR exceeds this value, it indicates high volatility and triggers a buy or sell signal.'
        }
    },
    "MovingAverageConvergenceDivergence": {
        'fast_period': {
            'value': 50,
            'description': 'The number of periods for calculating the fast moving average in the Moving Average Convergence Divergence (MACD) strategy. It determines the lookback period for the fast line.'
        },
        'slow_period': {
            'value': 150,
            'description': 'The number of periods for calculating the slow moving average in the MACD strategy. It determines the lookback period for the slow line.'
        },
        'signal_period': {
            'value': 50,
            'description': 'The number of periods for calculating the signal line in the MACD strategy. It determines the lookback period for the signal line.'
        }
    },
    "MovingAverageCrossover": {
        'short_period': {
            'value': 50,
            'description': 'The number of periods for calculating the short-term moving average in the Moving Average Crossover strategy. It determines the lookback period for the short-term moving average.'
        },
        'long_period': {
            'value': 200,
            'description': 'The number of periods for calculating the long-term moving average in the Moving Average Crossover strategy. It determines the lookback period for the long-term moving average.'
        }
    },
    "RelativeStrengthIndex": {
        'rsi_period': {
            'value': 21,
            'description': 'The number of periods for calculating the Relative Strength Index (RSI). It determines the lookback period for calculating RSI.'
        },
        'sell_threshold': {
            'value': 75,
            'description': 'The threshold value for the RSI to trigger selling. If the RSI crosses above this value, it indicates an overbought condition and triggers selling.'
        },
        'buy_threshold': {
            'value': 40,
            'description': 'The threshold value for the RSI to trigger buying. If the RSI crosses below this value, it indicates an oversold condition and triggers buying.'
        }
    },
    "StochasticOverboughtOversold": {
        'stoch_period': {
            'value': 14,
            'description': 'The number of periods for calculating the Stochastic Oscillator in the Stochastic Overbought/Oversold strategy. It determines the lookback period for the stochastic calculations.'
        },
        'stoch_threshold_oversold': {
            'value': 45,
            'description': 'The threshold value for the Stochastic Oscillator to trigger buying in the Stochastic Overbought/Oversold strategy. If the Stochastic Oscillator falls below this value and then crosses above it, it indicates an oversold condition and triggers buying.'
        },
        'stoch_threshold_overbought': {
            'value': 80,
            'description': 'The threshold value for the Stochastic Oscillator to trigger selling in the Stochastic Overbought/Oversold strategy. If the Stochastic Oscillator rises above this value and then crosses below it, it indicates an overbought condition and triggers selling.'
        }
    },
    "Turtle": {
        'entry_lookback': {
            'value': 10,
            'description': 'The number of periods for calculating the highest high and lowest low in the Turtle strategy. It determines the lookback period for identifying potential entry points.'
        },
        'exit_lookback': {
            'value': 20,
            'description': 'The number of bars to elapse before checking for exit conditions in the Turtle strategy. It determines the period for checking if prices cross back below the lowest low or above the highest high.'
        }
    },
    "VolatilityBreakout": {
        'lookback_period': {
            'value': 20,
            'description': 'The number of periods for calculating the highest high and lowest low in the Volatility Breakout strategy. It determines the lookback period for identifying breakouts based on recent volatility.'
        },
        'volatility_factor': {
            'value': 1.5,
            'description': 'The factor by which the highest high and lowest low are adjusted in the Volatility Breakout strategy. It determines the threshold for triggering buy or sell signals based on breakout levels adjusted by recent volatility.'
        },
        'stop_loss_percentage': {
            'value': 0.02,
            'description': 'The percentage below the entry price for setting the stop-loss level in the Volatility Breakout strategy. It determines the maximum acceptable loss for a trade.'
        },
        'take_profit_percentage': {
            'value': 0.02,
            'description': 'The percentage above the entry price for setting the take-profit level in the Volatility Breakout strategy. It determines the target profit level for a trade.'
        }
    }
}

### Insertion of the strategy info in MongoDB

In [24]:
# Create a list to store strategy data
strategy_data_list = []

# Iterate over the strategy classes
for i, (strategy_name, strategy_class) in enumerate(strategy_classes.items()):
    # Get the description of the strategy
    strategy_description = strategy_descriptions[strategy_name]

    # Get the parameters and descriptions of the strategy
    strategy_parameters_description = parameter_descriptions[strategy_name]

    # Create strategy data dictionary
    strategy_data = {
        'users': "all",
        'strategy_id': strategy_name,
        'description': strategy_description,
        'parameters': strategy_parameters_description,
    }

    # Append strategy data to the list
    strategy_data_list.append(strategy_data)

# Perform bulk insert
db['default_strategies'].insert_many(strategy_data_list)

In [16]:
for i, (module_name, strategy_class) in enumerate(strategy_classes.items()):
    print(i, (module_name, strategy_class))

0 ('AverageDirectionalMovement', <class 'modules.AverageDirectionalMovement.AverageDirectionalMovement'>)
1 ('BollingerBands', <class 'modules.BollingerBands.BollingerBands'>)
2 ('MeanReversion', <class 'modules.MeanReversion.MeanReversion'>)
3 ('MeanReversionBollinger', <class 'modules.MeanReversionBollinger.MeanReversionBollinger'>)
4 ('MomentumVolatility', <class 'modules.MomentumVolatility.MomentumVolatility'>)
5 ('MovingAverageConvergenceDivergence', <class 'modules.MovingAverageConvergenceDivergence.MovingAverageConvergenceDivergence'>)
6 ('MovingAverageCrossover', <class 'modules.MovingAverageCrossover.MovingAverageCrossover'>)
7 ('RelativeStrengthIndex', <class 'modules.RelativeStrengthIndex.RelativeStrengthIndex'>)
8 ('StochasticOverboughtOversold', <class 'modules.StochasticOverboughtOversold.StochasticOverboughtOversold'>)
9 ('Turtle', <class 'backtesting.backtesting.Strategy'>)
10 ('VolatilityBreakout', <class 'backtesting.backtesting.Strategy'>)


# END